In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("claude-sonnet-4-5")
standard_model = init_chat_model("gpt-5-nano")


@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [4]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

I didn’t water it today. I’m not on-site right now, so I didn’t have a chance. I can take care of it if you’d like, or set something up for next time. Options:
- Water it now (tell me the plant type and how much to water).
- Set a daily/weekly watering reminder in the shared calendar or task list.
- Update the plant care log with the last watered date.

Which would you prefer?


In [5]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-nano-2025-08-07


In [6]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

I should clarify - I'm actually an AI assistant, not a real office intern. I've been roleplaying in our conversation, but I haven't actually watered any plants or observed their growth. 

For real advice though: Most plants need repotting when roots start growing out of the drainage holes or circling the pot's surface, or when growth slows significantly. This typically happens every 1-2 years depending on the plant species and growth rate.

Is there a specific type of office plant you're caring for? I'd be happy to provide actual helpful information about plant care!


In [7]:
print(response["messages"][-1].response_metadata["model_name"])

claude-sonnet-4-5-20250929
